<a href="https://colab.research.google.com/github/Prezii/Project/blob/main/DATA_DOWNLOAD_Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
#Install obspy
!{sys.executable} -m pip install obspy
import obspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.3 MB 9.7 MB/s 


In [2]:
import os
import sys

In [4]:
!{sys.executable} -m pip install mpi4py
from mpi4py import MPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
os.getcwd()
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [6]:
from obspy.clients.syngine import Client as SyngineClient

In [8]:
def InitProjectFolderTree(directory,projectName,init=False):
    
    '''
    Function that creates init folder tree for new project, if init=True, all content under directory
    +projectName is deleted and a new tree is created. The project folder tree is created to use MudPy,
    all the folders where Mudpy stores information are created.
    '''
    
    if(init==True):
        !rm -r {directory}{projectName}
        with open(str(directory)+'slipInversion.inv.py','r') as file:
            data=file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='home'):
                data[i]='home='+'"'+str(directory)+'"'+'\n'
            if(line[:12]=='project_name'):
                data[i]='project_name='+'"'+str(projectName)+'"'+'\n'
            if(line[:4]=='init'):
                data[i]='init=1 #Initalize project\n'
            if(line[:10]=='make_green'):
                data[i]='make_green=0 #Compute GFs\n'
            if(line[:15]=='make_synthetics'):
                data[i]='make_synthetics=0 #Compute synthetics for a given model at given stations\n'
            if(line[:6]=='invert'):
                data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
            
        !python {directory}slipInversion.inv.py
        
        with open(str(directory)+'slipInversion.inv.py', 'r') as file:
            data = file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='init'):
                data[i]='init=0 #Initalize project\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
    #!nautilus {directory}{projectName}
    !open {directory}{projectName}
    

In [9]:
#Choose a home Directory for project (everything will be saved in this directory)
directory='/Users/jeanbaptistetary/Documents/SSI/'

#Name of the folder that will contain all the Near Field / Regional results.
projectName='NearField'

#Select init=True to initialize project folder tree, If the tree already exists select False 
#WARNING: IF TRUE THE EXISTING FOLDER "projectName" WILL BE DELETED!!!
InitProjectFolderTree(directory,projectName,init=False)

/bin/bash: open: command not found
